# Rank and Partition

Bu challenge'ın amacı, `RANK` veya `SUM` gibi Window Functions kullanmaktır (bkz. [Window Functions](https://mode.com/sql-tutorial/sql-window-functions/)) ve bunları `OVER` keyword'ü ile uygulamaktır.

`OVER` clause şu şekilde kullanılabilir:
- Satırların kapsamını seçmek için `PARTITION BY`
- Window function'ın uygulanacağı satırların sıralamasını belirlemek için `ORDER BY`


## Data
Bu URL'de bulunan `ecommerce.sqlite` database ile çalışacağız:
`https://wagon-public-datasets.s3.amazonaws.com/sql_databases/ecommerce.sqlite`

Dosyayı indirmek için aşağıdaki hücreyi çalıştırın:

In [1]:
!curl https://wagon-public-datasets.s3.amazonaws.com/sql_databases/ecommerce.sqlite > data/ecommerce.sqlite

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9216  100  9216    0     0  11584      0 --:--:-- --:--:-- --:--:-- 11577


## Database Discovery

Bu database'i daha önce keşfettiniz. Database'i indirdikten sonra, VS Code SQLite extension'ını kullanarak tekrar inceleyebileceğinizi unutmayın.

- `Ctrl-Shift-P` veya `Cmd-Shift-P` tuşlarına basın
- `SQLite: Open Database` görünene kadar yazmaya başlayın
- `Enter`
- Açılan listeden database dosyasını seçin
- Sol alt köşede `SQLITE EXPLORER` butonuna tıklayın


## Setup

Pandas ve sqlite3 ihtiyacımız olan her şey :-)

In [2]:
import pandas as pd
from sqlite3 import connect

## Order rank per customer

👉 Her müşteri için order date'e göre orders kayıtlarını sıralayan bir SQL Query yazın.

Her müşteri için orders kronolojik olarak sıralanmalıdır. `OrderID`, `CustomerID`, `OrderDate`, `OrderRank` döndürün.

In [7]:
# Her müşterinin siparişlerini sipariş tarihine göre sıralayın.
# Her müşteri için siparişler kronolojik sırayla sıralanmalıdır.
# `OrderID`, `CustomerID`, `OrderDate`, `OrderRank` değerlerini return edin.
query_order_rank_per_customer = """
  SELECT 
        OrderID, 
        CustomerID, 
        OrderDate,
        RANK() OVER (
            PARTITION BY CustomerID 
            ORDER BY OrderDate ASC
        ) AS OrderRank
    FROM Orders
"""

In [8]:
with connect('data/ecommerce.sqlite') as conn:
    df = pd.read_sql(
        query_order_rank_per_customer,
        con=conn
    )
df.head()

,OrderID,CustomerID,OrderDate,OrderRank
0,1,1,2012-01-04,1
1,8,1,2012-06-13,2
2,12,1,2012-09-13,3
3,2,2,2012-01-27,1
4,4,2,2012-03-13,2


Sonuç beklediğiniz gibi göründüğünde, sorgunuzu denemek ve test etmek için aşağıdaki hücreleri çalıştırın.

In [9]:
from nbresult import ChallengeResult
result = ChallengeResult(
    'order_rank_per_customer',
    query=query_order_rank_per_customer
)
result.write(); print(result.check())

## Order cumulative amount per customer

👉 Her müşteri için order date’e göre orders kayıtlarının kümülatif amount (USD cinsinden) değerini hesaplayan bir SQL Query yazın.

Her müşteri için orders kronolojik olarak sıralanmalıdır. `OrderID`, `CustomerID`, `OrderDate`, `OrderCumulativeAmount` döndürün.

In [10]:
# Sipariş tarihine göre her müşterinin siparişlerinin toplam tutarını (USD cinsinden) döndürün.
# Her müşteri için siparişler kronolojik sırayla sıralanmalıdır.
# `OrderID`, `CustomerID`, `OrderDate`, `OrderCumulativeAmount` değerlerini return edin.
query_order_cumulative_amount_per_customer = """
    SELECT 
        OrderID, 
        CustomerID, 
        OrderDate,
        SUM(TotalAmount) OVER (
            PARTITION BY CustomerID 
            ORDER BY OrderDate ASC
        ) AS OrderCumulativeAmount
    FROM Orders
"""

In [11]:
with connect('data/ecommerce.sqlite') as conn:
    df = pd.read_sql(
        query_order_cumulative_amount_per_customer,
        con=conn
    )
df.head()

DatabaseError: Execution failed on sql '
    SELECT 
        OrderID, 
        CustomerID, 
        OrderDate,
        SUM(TotalAmount) OVER (
            PARTITION BY CustomerID 
            ORDER BY OrderDate ASC
        ) AS OrderCumulativeAmount
    FROM Orders
': no such column: TotalAmount

Sonuç beklediğiniz gibi göründüğünde, sorgunuzu denemek ve test etmek için aşağıdaki hücreleri çalıştırın.

In [ ]:
from nbresult import ChallengeResult
result = ChallengeResult(
    'order_cumulative_amount_per_customer',
    query=query_order_cumulative_amount_per_customer
)
result.write(); print(result.check())

## Key learning points

- Window functions, satırları **aggregate etmeden** farklı satırlardaki verileri hesaplamanıza olanak tanır.
- Satırları aggregate etmek istiyorsanız `GROUP BY` kullanın.
- Eğer aggregate etmek istemiyorsanız, bunun yerine `OVER` ile `PARTITION BY` ve `ORDER BY` kullanın.